# EDA on Chase Bank Branch Deposits

# Content

This dataset includes a record for every branch of Chase Bank in the United States, including the branch's name and number, date established as a bank office and (if applicable) acquired by JP Morgan Chase, physical location as street address, city, state, zip, and latitude and longitude coordinates, and the amount deposited at the branch (or the institution, for the bank's main office) between July 1 and June 30, 2016, in US dollars.


# Data Source

The location data was scraped from the Chase Bank website. The deposit data was compiled from the Federal Deposit Insurance Corporation's annual Summary of Deposits reports. This perticular data was taken from the Kaggle Link :- https://www.kaggle.com/chasebank/bank-deposits


# EDA Ideas

* Where did Chase Bank customers deposit the most money last year?
* Which bank branch has seen the most growth in deposits?
* How did the bank network of branch locations grow over the past century?
* What city has the most bank branches per capita?

In [1]:
import numpy as np # linear algebra
import pandas_profiling
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #for Plotting
%matplotlib inline
import seaborn as sns #for graphing
local_path = "C:/Users/Aravind/Desktop/UpX Academy/Data Science Track - 01/PROJECTS TO BE DOING/Project Datasets/Project_Bank/Chase bank-deposits/database.csv"
input_data = pd.read_csv(local_path)                           #load CSV file from local directory
input_data.head()                                         # Preview the first 5 lines of the loaded data

,Institution Name,Main Office,Branch Name,Branch Number,Established Date,Acquired Date,Street Address,City,County,State,Zipcode,Latitude,Longitude,2010 Deposits,2011 Deposits,2012 Deposits,2013 Deposits,2014 Deposits,2015 Deposits,2016 Deposits
0,JPMorgan Chase Bank,1,JPMorgan Chase Bank Main Office,0,01/01/1824,NaN,1111 Polaris Parkway,Columbus,Delaware,OH,43240,40.14453,-82.99115,633131000.0,743268000.0,832455000.0,916543000.0,1.032549e+09,1.069425e+09,1155185000
1,JPMorgan Chase Bank,0,Vernon Hills Scarsdale Branch,2,03/20/1961,NaN,676 White Plains Road,Scarsdale,Westchester,NY,10583,40.97008,-73.80670,293229.0,310791.0,325742.0,327930.0,3.277920e+05,3.414750e+05,381558
2,JPMorgan Chase Bank,0,Great Neck Northern Boulevard Branch,3,09/09/1963,NaN,410 Northern Boulevard,Great Neck,Nassau,NY,11021,40.77944,-73.72240,191011.0,206933.0,216439.0,237983.0,2.341830e+05,2.624550e+05,278940
3,JPMorgan Chase Bank,0,North Hartsdale Branch,4,02/19/1966,NaN,353 North Central Avenue,Hartsdale,Westchester,NY,10530,41.02654,-73.79168,87110.0,88367.0,93163.0,109659.0,1.119850e+05,1.167720e+05,140233
4,JPMorgan Chase Bank,0,Lawrence Rockaway Branch,5,01/16/1965,NaN,335 Rockaway Turnpike,Lawrence,Nassau,NY,11559,40.62715,-73.73675,172608.0,172749.0,189413.0,198445.0,2.051980e+05,2.232000e+05,235594


In [ ]:
eda = pandas_profiling.ProfileReport(input_data)
eda

In [ ]:
df.groupby('Branch Name').size().count()
# Here we can see there are total 5317 branches present.

In [3]:
df = pd.DataFrame(input_data)          
input_data.shape

(5413, 20)

### 1) Where did Chase Bank customers deposit the most money last year?

In [29]:
df_city = df.pivot(columns = 'City', values = '2016 Deposits').sum()
df_city
df_county = df.pivot(columns = 'County', values = '2016 Deposits').sum()
df_county
df_state = df.pivot(columns = 'State', values = '2016 Deposits').sum()
df_state
df_branch_name = df.pivot(columns = 'Branch Name', values = '2016 Deposits').sum()
df_branch_name

Branch Name
1 30 and Broadway Branch                          32264.0
112th & Broadway Branch                          139455.0
12 Mile-Country Club Drive Branch                168302.0
12335 Venice Blvd Branch                         175831.0
135th Street & Fifth Avenue Branch                67947.0
1400 NW 17th Avenue Branch                        75874.0
1600 N. Plano Rd Branch                           49539.0
187-15 Union Turnpike Branch                     167383.0
1870 Aloma Branch                                 56265.0
2209 Via Anacapa Branch                           91958.0
225th & Broadway Branch                           22200.0
23402 W Lyons Ave Branch                         104906.0
23rd & Jackson Financial Ctr Branch               35659.0
24 Mile-Shelby Road Branch                       132564.0
241 W Main Branch                                145323.0
27th and Loomis Branch                            72966.0
290 and Spring Cypress Banking Center Branch      92722.0
29

In [ ]:
# Initialize the matplotlib figure
import matplotlib.pyplot as plt
import seaborn as sns
f, ax = plt.subplots(figsize=(6, 15))
city = sns.load_dataset('df_city')
sns.set(style ='whitegrid')
sns.axes_style = ('whitegrid')
city_bar = sns.barplot(x = '2016 Deposits', y = 'City', data = df_city)

In [11]:
df.isnull().sum(axis=0)         #Identifying number of Nan Values

Institution Name       0
Main Office            0
Branch Name            0
Branch Number          0
Established Date       0
Acquired Date       1615
Street Address         0
City                   0
County                 0
State                  0
Zipcode                0
Latitude              66
Longitude             66
2010 Deposits        740
2011 Deposits        578
2012 Deposits        329
2013 Deposits        175
2014 Deposits         56
2015 Deposits         19
2016 Deposits          0
dtype: int64

In [ ]:
# Since Latitude, Longitude are not required for the purpose of this EDA, we shall remove those columns
df.drop(['Latitude','Longitude'], axis = 1, inplace = True)
df

### 2) Which bank branch has seen the most growth in deposits?

In [96]:
df_2010_11 = pd.DataFrame(df, columns = ['Branch Name', '2010 Deposits', '2011 Deposits'])
df_2010_11['diff_2010_11'] = df['2010 Deposits'] - df['2011 Deposits']
df_2010_11['diff_2010_11']

0      -110137000.0
1          -17562.0
2          -15922.0
3           -1257.0
4            -141.0
5           10148.0
6           -5041.0
7           -2658.0
8            3653.0
9           -3038.0
10          -7208.0
11          -5885.0
12           1500.0
13          -2107.0
14          -8171.0
15          -1258.0
16          -5055.0
17           6691.0
18          -6924.0
19          -5839.0
20          19602.0
21         -34170.0
22         -25701.0
23         -10003.0
24         -42860.0
25         -26791.0
26         -39166.0
27         -10672.0
28         -20671.0
29           4215.0
           ...     
5383            NaN
5384            NaN
5385            NaN
5386            NaN
5387            NaN
5388            NaN
5389            NaN
5390            NaN
5391            NaN
5392            NaN
5393            NaN
5394            NaN
5395            NaN
5396            NaN
5397            NaN
5398            NaN
5399            NaN
5400            NaN
5401            NaN


In [ ]:
df_city
df_2010_11 = pd.DataFrame(df, columns= ['2010 Deposits','2011 Deposits'])
df_2010_11_pct = ((df_2010_11['2011 Deposits'] - df_2010_11['2010 Deposits'])/(df_2010_11['2010 Deposits']))*100
input_data[['Branch Name']] + df_2010_11_pct 

### 3) How did the bank network of branch locations grow over the past century?


In [ ]:
df_city = df.pivot(columns = 'Branch Name', values = ('2010 Deposits', '2011 Deposits')).sum()
df_city

### 4) What city has the most bank branches per capita?